In [1]:
pip install pandas numpy openpyxl


In [7]:
import pandas as pd

# Upload and read the Excel file
uploaded_file_path = "solid_block2.xlsx"  # Update this with your actual file path
inspection_data = pd.read_excel(uploaded_file_path, sheet_name="Solid Blocks")  # Reading inspection data
rules_data = pd.read_excel(uploaded_file_path, sheet_name="Rules")  # Reading rules

# Step 1: Create the dictionary from the Solid Blocks sheet
reference_dimensions = {}
for _, row in inspection_data.iterrows():
    block_type = row["Block Type Name"]
    reference_dimensions[block_type] = {
        "Length": row["Reference Length"],
        "Width": row["Reference Width"],
        "Height": row["Reference Height"]
    }

# Print the created reference_dimensions dictionary
print("Reference Dimensions Dictionary:")
print(reference_dimensions)

# Step 2: Add reference dimensions to the inspection data using the dictionary
inspection_data["Reference Length"] = inspection_data["Block Type Name"].map(
    lambda x: reference_dimensions.get(x, {}).get("Length", None)
)
inspection_data["Reference Width"] = inspection_data["Block Type Name"].map(
    lambda x: reference_dimensions.get(x, {}).get("Width", None)
)
inspection_data["Reference Height"] = inspection_data["Block Type Name"].map(
    lambda x: reference_dimensions.get(x, {}).get("Height", None)
)

# Step 3: Define tolerances from the rules (e.g., ±5 mm for Length, Width, Height)
length_tolerance = 5
width_tolerance = 5
height_tolerance = 5

# Step 4: Check if the inspected dimensions are within the tolerances
inspection_data["Length Status"] = inspection_data.apply(
    lambda row: "Pass" if abs(row["Actual Length"] - row["Reference Length"]) <= length_tolerance else "Fail", axis=1
)
inspection_data["Width Status"] = inspection_data.apply(
    lambda row: "Pass" if abs(row["Actual Width"] - row["Reference Width"]) <= width_tolerance else "Fail", axis=1
)
inspection_data["Height Status"] = inspection_data.apply(
    lambda row: "Pass" if abs(row["Actual Height"] - row["Reference Height"]) <= height_tolerance else "Fail", axis=1
)

# Step 5: Final material pass/fail based on all dimensions
inspection_data["Material Status"] = inspection_data.apply(
    lambda row: "Pass" if row["Length Status"] == "Pass" and row["Width Status"] == "Pass" and row["Height Status"] == "Pass" else "Fail", axis=1
)

# Display the final processed data
print("\nProcessed Inspection Data:")
print(inspection_data)

# Export the results to a new Excel file
output_file_path = "/path_to_save_processed_data.xlsx"  # Specify the output path
inspection_data.to_excel(output_file_path, index=False)
print(f"Processed data saved to {output_file_path}")



Reference Dimensions Dictionary:
{nan: {'Length': nan, 'Width': nan, 'Height': nan}, '400x150x200 (6″)': {'Length': 400.0, 'Width': 150.0, 'Height': 200.0}, '400x100x200 (4″)': {'Length': 400.0, 'Width': 100.0, 'Height': 200.0}, '400x200x200 (8″)': {'Length': 400.0, 'Width': 200.0, 'Height': 200.0}}

Processed Inspection Data:
    MIR No          Entry Date Checked By Type of Material  \
0      NaN                 NaT        NaN              NaN   
1   3020.0 2024-11-14 11:50:00    gowtham     Solid Blocks   
2   3027.0 2024-11-15 10:50:00    gowtham     Solid Blocks   
3   3030.0 2024-11-15 12:45:00    gowtham     Solid Blocks   
4   3035.0 2024-11-16 14:35:00    gowtham     Solid Blocks   
5   3040.0 2024-11-18 14:20:00    gowtham     Solid Blocks   
6   3046.0 2024-11-19 12:48:00    gowtham     Solid Blocks   
7   3051.0 2024-11-20 09:45:00    gowtham     Solid Blocks   
8   3054.0 2024-11-20 12:30:00    gowtham     Solid Blocks   
9   3055.0 2024-11-20 12:50:00    gowtham     Solid

In [8]:
pip install openai


In [ ]:
import openai
import pandas as pd

# Load processed data
inspection_data = pd.read_excel("solid_block2.xlsx")  # Replace with your actual file path

# Set OpenAI API Key
openai.api_key = "insert"  # Replace with your actual OpenAI API key

# Define a function to handle queries
def query_inspection_data(question):
    # Convert the inspection data to a string format for the model
    data_context = inspection_data.to_string(index=False)

    # Ensure that the query is asking for material pass/fail counts
    if "how many materials passed inspection" in question.lower():
        # Count the number of materials that passed inspection
        pass_count = inspection_data[inspection_data['Status'] == 'Pass'].shape[0]
        return f"Number of materials that passed inspection: {pass_count}"

    # If it's not a pass/fail query, continue with the normal chat functionality
    messages = [
        {"role": "system", "content": "You are an intelligent assistant analyzing material inspection data."},
        {"role": "user", "content": f"The data is as follows:\n{data_context}\n\nQuestion: {question}"}
    ]

    # Use OpenAI's ChatGPT model
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # You can also use "gpt-4" if available
        messages=messages,
        max_tokens=150,  # Limit response size
        temperature=0,   # Controls randomness (0 = deterministic)
    )

    # Extract and return the response
    return response.choices[0].message.content.strip()

# Example queries
#question = "How many materials passed inspection?"
#answer = query_inspection_data(question)
#print("Q:", question)
#print("A:", answer)


**UI Design**

In [24]:
#!pip install streamlit
!pip install gradio


In [25]:
# import streamlit as st
# # Streamlit UI Setup
# st.title("Material Inspection Data Query Assistant")
# st.write("Ask a question about the material inspection data.")
#
# # Input for user question
# user_question = st.text_input("Enter your question:")
#
# if user_question:
#     # Get the answer to the user's question
#     answer = query_inspection_data(user_question)
#
#     # Display the result
#     st.write(f"**Answer**: {answer}")

import gradio as gr

# Create Gradio interface
def gradio_query_interface(user_question):
    try:
        answer = query_inspection_data(user_question)
        return answer
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Set up the Gradio interface
iface = gr.Interface(
    fn=gradio_query_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question about material inspection data..."),
    outputs="text",
    title="Material Inspection Data Query Assistant",
    description="Ask questions about the material inspection data in natural language.",
    examples=[
        ["How many materials passed inspection?"],
        ["What are the reference dimensions?"],
        ["Show me details of failed materials"],
        ["What is the tolerance for block dimensions?"]
    ]
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01c40dbecde1c4869d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
!pip freeze > requirements.txt
